In [1]:
# =======================================================================
# Snapdragon X Elite Optimization Configuration
# =======================================================================


# =======================================================================
# Environment & Optimizations
# =======================================================================
import os
import time
import numpy as np
import tensorflow as tf

# Threading and backend opts (tune "8" to your CPU)
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "1"
os.environ["OMP_NUM_THREADS"] = "8"
os.environ["TF_NUM_INTRAOP_THREADS"] = "8"
os.environ["TF_NUM_INTEROP_THREADS"] = "8"



# Import deep learning libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2


# =======================================================================
# Configuration Parameters
# =======================================================================

picture_size = 48  # Input image size (48x48 grayscale)
batch_size = 128
epochs = 30
no_of_classes = 7  # Angry, Disgust, Fear, Happy, Neutral, Sad, Surprise
learning_rate = 0.0001

train_dir = 'data/images/train/'
val_dir   = 'data/images/validation/'

print(f"Image size: {picture_size}x{picture_size} grayscale")
print(f"Batch size: {batch_size}")
print(f"Epochs: {epochs}")


# 1. Enable XLA (Accelerated Linear Algebra) compilation
tf.config.optimizer.set_jit(True)
print("✅ XLA compilation enabled")

# 2. Configure thread pools for ARM big.LITTLE architecture
# Snapdragon X Elite has performance and efficiency cores
tf.config.threading.set_inter_op_parallelism_threads(8)  # Between operations
tf.config.threading.set_intra_op_parallelism_threads(8)  # Within operations
print("✅ Multi-threading optimized for ARM architecture")

# 3. Enable oneDNN optimizations (ARM compute library)
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'

# 4. Memory growth to prevent OOM on ARM devices
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print(f"✅ Found {len(physical_devices)} GPU/NPU device(s)")


Image size: 48x48 grayscale
Batch size: 128
Epochs: 30
✅ XLA compilation enabled
✅ Multi-threading optimized for ARM architecture


In [7]:

# 5. Mixed precision (already in your code - keep it!)
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print("✅ Mixed precision FP16 enabled")

# GPU/NPU memory growth (if present)
for dev_type in ['GPU', 'TPU']:
    devices = tf.config.list_physical_devices(dev_type)
    if devices:
        try:
            for d in devices:
                tf.config.experimental.set_memory_growth(d, True)
            print(f"✅ Memory growth set for {dev_type}")
        except Exception as e:
            print(f"⚠️ Could not set memory growth for {dev_type}: {e}")    



# 6. Prefetch and cache dataset for faster loading
def optimize_dataset(generator, cache=True):
    """Optimize data pipeline for Snapdragon NPU"""
    AUTOTUNE = tf.data.AUTOTUNE
    
    # Convert generator to tf.data.Dataset
    dataset = tf.data.Dataset.from_generator(
        lambda: generator,
        output_signature=(
            tf.TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 7), dtype=tf.float32)
        )
    )
    
    if cache:
        dataset = dataset.cache()  # Cache in memory
    
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)  # Prefetch next batch
    
    return dataset


✅ Mixed precision FP16 enabled


In [8]:
# =======================================================================
# Optimized Model Architecture for Snapdragon NPU
# =======================================================================

def build_optimized_model():
    """
    Lightweight model optimized for Snapdragon X Elite NPU.
    Uses depthwise separable convolutions (NPU-friendly).
    """
    from tensorflow.keras.layers import DepthwiseConv2D, SeparableConv2D
    
    inputs = Input(shape=(picture_size, picture_size, 1))
    
    # Initial conv
    x = Conv2D(32, (3, 3), padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Depthwise separable blocks (NPU optimized)
    def depthwise_block(x, filters, strides=1):
        x = SeparableConv2D(filters, (3, 3), strides=strides, 
                           padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        return x
    
    x = depthwise_block(x, 64, strides=2)   # 24x24
    x = depthwise_block(x, 128, strides=2)  # 12x12
    x = depthwise_block(x, 256, strides=2)  # 6x6
    x = depthwise_block(x, 512)             # 6x6
    
    # Global pooling and classification
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(7, activation='softmax', dtype='float32')(x)
    
    model = Model(inputs, outputs)
    
    return model


# =======================================================================
# Build Model (choose ONE option)
# =======================================================================
option = 1

if option == 1:
# OPTION 1: Use your existing MobileNetV2 model (keep your current code)
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(48, 48, 3)
    )

    base_model.trainable = False

    inputs = Input(shape=(48, 48, 1))
    x = tf.keras.layers.Concatenate(axis=-1)([inputs, inputs, inputs])
    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(7, activation='softmax', dtype='float32')(x)

    model = Model(inputs, outputs)

# OR

# OPTION 2: Use the lightweight optimized model (faster on Snapdragon)
elif option == 2:
    model = build_optimized_model()

print(f"Total parameters: {model.count_params():,}")




C:\Users\marty\AppData\Local\Temp\ipykernel_14240\3056608763.py:52: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


Total parameters: 2,621,255


In [9]:

# =======================================================================
# Quantization-Aware Training (for NPU deployment)
# =======================================================================

def convert_to_tflite_quantized(model, train_generator, output_path='emotion_model_int8.tflite'):
    train_generator.reset()
    """
    Convert model to INT8 quantized TFLite for maximum NPU performance.
    This will run MUCH faster on Snapdragon NPU.
    """
    import tensorflow as tf
    
    # Representative dataset for quantization
    def representative_dataset():
        for i in range(100):
            batch, _ = next(iter(train_generator))
            yield [batch[:1].astype(np.float32)]
    
    # Convert to TFLite with INT8 quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    
    tflite_model = converter.convert()
    
    # Save quantized model
    with open(output_path, 'wb') as f:
        f.write(tflite_model)
    
    print(f"✅ INT8 quantized model saved: {output_path}")
    print(f"   Model size: {len(tflite_model) / 1024:.2f} KB")
    print("   This model will run 3-5x faster on Snapdragon NPU!")
    
    return tflite_model


In [10]:
# =======================================================================
# Data loaders
# =======================================================================
# Simple normalization; you can add augmentations as needed
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen_val = ImageDataGenerator(rescale=1./255)

train_set = datagen_train.flow_from_directory(
    train_dir,
    target_size=(picture_size, picture_size),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_set = datagen_val.flow_from_directory(
    val_dir,
    target_size=(picture_size, picture_size),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

print(f"Training samples: {train_set.n} | Validation samples: {val_set.n}")
print(f"Classes: {train_set.class_indices}")

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
Training samples: 28821 | Validation samples: 7066
Classes: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [11]:
# =======================================================================
# Compile (avoid jit_compile=True here for stability on ARM + FP16)
# =======================================================================

# =======================================================================
# Now compile
# =======================================================================

opt = tf.keras.optimizers.Adam(
    learning_rate=learning_rate,
    clipnorm=1.0,
    epsilon=1e-7
)

model.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
    #jit_compile=True
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 48, 48, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 48, 48, 3) │          0 │ input_layer_1[0]… │
│ (Concatenate)       │                   │            │ input_layer_1[0]… │
│                     │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mobilenetv2_1.00_2… │ (None, 2, 2,      │  2,257,984 │ concatenate[0][0] │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ mobilenetv2_1.00… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    327,936 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256)       │      1,024 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 7)         │        903 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,621,255 (10.00 MB)

 Trainable params: 362,503 (1.38 MB)

 Non-trainable params: 2,258,752 (8.62 MB)

In [12]:
# =======================================================================
# Callbacks
# =======================================================================
checkpoint = ModelCheckpoint(
    filepath="best_model.keras",
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    verbose=1,
    min_lr=1e-5
)

class PerformanceCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()
    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start
        steps = train_set.n // train_set.batch_size
        imgs = steps * train_set.batch_size
        sps = imgs / epoch_time if epoch_time > 0 else 0.0
        print(f"\n⚡ Epoch time: {epoch_time:.1f}s | Throughput: {sps:.0f} img/s")

callbacks = [checkpoint, early_stopping, reduce_lr, PerformanceCallback()]

In [13]:
# =======================================================================
# Train
# =======================================================================
print("\n🚀 Starting training...")
history = model.fit(
    train_set,
    steps_per_epoch=train_set.n // train_set.batch_size,
    validation_data=val_set,
    validation_steps=val_set.n // val_set.batch_size,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1
)
print("✅ Training complete")

# Optional: fine-tune some layers for better accuracy
# Unfreeze last N blocks if you have time/headroom
# base_model.trainable = True
# for layer in base_model.layers[:-40]:
#     layer.trainable = False
# model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
# history_ft = model.fit(...)


🚀 Starting training...


c:\Users\marty\anaconda3\envs\tf311_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 777ms/step - accuracy: 0.1730 - loss: 2.7038
Epoch 1: val_accuracy improved from None to 0.29205, saving model to best_model.keras

⚡ Epoch time: 232.9s | Throughput: 124 img/s
225/225 ━━━━━━━━━━━━━━━━━━━━ 233s 980ms/step - accuracy: 0.1907 - loss: 2.5837 - val_accuracy: 0.2920 - val_loss: 1.9158 - learning_rate: 1.0000e-04
Epoch 2/30
  1/225 ━━━━━━━━━━━━━━━━━━━━ 2:53 774ms/step - accuracy: 0.1562 - loss: 2.3705

c:\Users\marty\anaconda3\envs\tf311_env\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy improved from 0.29205 to 0.29233, saving model to best_model.keras

⚡ Epoch time: 82.0s | Throughput: 351 img/s
225/225 ━━━━━━━━━━━━━━━━━━━━ 82s 363ms/step - accuracy: 0.1562 - loss: 2.3705 - val_accuracy: 0.2923 - val_loss: 1.9151 - learning_rate: 1.0000e-04
Epoch 3/30
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 769ms/step - accuracy: 0.2191 - loss: 2.3590
Epoch 3: val_accuracy improved from 0.29233 to 0.32259, saving model to best_model.keras

⚡ Epoch time: 214.8s | Throughput: 134 img/s
225/225 ━━━━━━━━━━━━━━━━━━━━ 215s 955ms/step - accuracy: 0.2227 - loss: 2.3207 - val_accuracy: 0.3226 - val_loss: 1.8246 - learning_rate: 1.0000e-04
Epoch 4/30
  1/225 ━━━━━━━━━━━━━━━━━━━━ 2:49 757ms/step - accuracy: 0.2266 - loss: 2.3002
Epoch 4: val_accuracy did not improve from 0.32259

⚡ Epoch time: 41.6s | Throughput: 693 img/s
225/225 ━━━━━━━━━━━━━━━━━━━━ 42s 182ms/step - accuracy: 0.2266 - loss: 2.3002 - val_accuracy: 0.3226 - val_loss: 1.8244 - learning_rate: 1.0000e-04
Epoch 5/30


In [14]:
# =======================================================================
# Save (versioned)
# =======================================================================
def save_model_versioned(model, base_name='emotion_model', extension='.keras', directory='.', save_weights=True):
    import re, os
    existing = [f for f in os.listdir(directory) if re.match(rf'{base_name}_v\d+{re.escape(extension)}', f)]
    if existing:
        vers = [int(re.search(rf'{base_name}_v(\d+){re.escape(extension)}', f).group(1)) for f in existing]
        next_v = max(vers) + 1
    else:
        next_v = 1
    model_path = os.path.join(directory, f"{base_name}_v{next_v}{extension}")
    model.save(model_path)
    print(f"💾 Saved model: {model_path}")
    paths = {'model': model_path}
    if save_weights:
        weights_path = os.path.join(directory, f"{base_name}_v{next_v}.weights.h5")
        model.save_weights(weights_path)
        print(f"💾 Saved weights: {weights_path}")
        paths['weights'] = weights_path
    return paths

paths = save_model_versioned(model, base_name='emotion_model', save_weights=True)

💾 Saved model: .\emotion_model_v5.keras
💾 Saved weights: .\emotion_model_v5.weights.h5


In [15]:
# =======================================================================
# Export for Snapdragon deployment
# =======================================================================
print("\n" + "="*70)
print("📦 Exporting models for Snapdragon deployment")
print("="*70)

# 1) TFLite INT8 (best for NPU)
def convert_to_tflite_int8(model, train_generator, output_path='emotion_model_int8.tflite', reps=100):
    train_generator.reset()
    def representative_dataset():
        yielded = 0
        while yielded < reps:
            # Use a fresh iterator each time to avoid exhaustion issues
            batch, _ = next(iter(train_generator))
            for i in range(min(len(batch), reps - yielded)):
                yield [batch[i:i+1].astype(np.float32)]
                yielded += 1

    conv = tf.lite.TFLiteConverter.from_keras_model(model)
    conv.optimizations = [tf.lite.Optimize.DEFAULT]
    conv.representative_dataset = representative_dataset
    conv.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    conv.inference_input_type = tf.uint8
    conv.inference_output_type = tf.uint8
    tfl = conv.convert()
    with open(output_path, 'wb') as f:
        f.write(tfl)
    print(f"✅ INT8 TFLite saved: {output_path} ({len(tfl)/1024:.1f} KB)")

convert_to_tflite_int8(model, train_set, 'emotion_model_int8.tflite', reps=128)

# 2) TFLite FP16 (good balance)
conv = tf.lite.TFLiteConverter.from_keras_model(model)
conv.optimizations = [tf.lite.Optimize.DEFAULT]
conv.target_spec.supported_types = [tf.float16]
tfl_fp16 = conv.convert()
with open('emotion_model_fp16.tflite', 'wb') as f:
    f.write(tfl_fp16)
print(f"✅ FP16 TFLite saved: emotion_model_fp16.tflite ({len(tfl_fp16)/1024:.1f} KB)")

# 3) ONNX (optional)
try:
    import tf2onnx
    spec = (tf.TensorSpec((None, picture_size, picture_size, 1), tf.float32, name="input"),)
    _proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path="emotion_model.onnx")
    print("✅ ONNX saved: emotion_model.onnx")
except Exception as e:
    print(f"⚠️ ONNX export skipped: {e}")

print("="*70)
print("Done.")


📦 Exporting models for Snapdragon deployment
INFO:tensorflow:Assets written to: C:\Users\marty\AppData\Local\Temp\tmp5hm625wl\assets


INFO:tensorflow:Assets written to: C:\Users\marty\AppData\Local\Temp\tmp5hm625wl\assets


Saved artifact at 'C:\Users\marty\AppData\Local\Temp\tmp5hm625wl'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32, name='keras_tensor_155')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  2638993211472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2638993214736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639007856400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639007856016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639007855440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2638993215120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639007856976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639007856592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639007855632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639007855056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  263

c:\Users\marty\anaconda3\envs\tf311_env\Lib\site-packages\tensorflow\lite\python\convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


ConverterError: Could not translate MLIR to FlatBuffer.<unknown>:0: error: loc(callsite(callsite(fused["ConcatV2:", "functional_1/concatenate_1/concat@__inference_function_104153"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_105260"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): 'tf.ConcatV2' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"]): called from
<unknown>:0: note: loc(callsite(callsite(fused["ConcatV2:", "functional_1/concatenate_1/concat@__inference_function_104153"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_105260"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): see current operation: %171 = "tf.ConcatV2"(%170, %170, %170, %4) {device = ""} : (tensor<?x48x48x1xf16>, tensor<?x48x48x1xf16>, tensor<?x48x48x1xf16>, tensor<i32>) -> tensor<?x48x48x3xf16>
<unknown

============================== end ==========================

In [ ]:
# =======================================================================
# Model: MobileNetV2 feature extractor for grayscale
# =======================================================================
# MobileNetV2 expects 3-channel input; we replicate grayscale to RGB
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(picture_size, picture_size, 3)
)
base_model.trainable = False  # freeze for initial training

inputs = Input(shape=(picture_size, picture_size, 1))
x = tf.keras.layers.Concatenate(axis=-1)([inputs, inputs, inputs])  # to 3 channels
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
# Ensure numerically stable float32 at the output with mixed precision
outputs = Dense(no_of_classes, activation='softmax', dtype='float32')(x)

model = Model(inputs, outputs)
print(f"Total parameters: {model.count_params():,}")

In [ ]:
# Test data pipeline
print("\n🔍 Testing data pipeline...")
sample_batch, sample_labels = next(iter(train_set))
print(f"Batch shape: {sample_batch.shape}")
print(f"Batch dtype: {sample_batch.dtype}")
print(f"Batch range: [{sample_batch.min():.3f}, {sample_batch.max():.3f}]")
print(f"Labels shape: {sample_labels.shape}")

# Test model prediction with error handling
try:
    test_pred = model.predict(sample_batch[:1], verbose=0)
    print(f"Prediction shape: {test_pred.shape}")
    print(f"Prediction values: {test_pred[0]}")
    print("✅ Data pipeline working!\n")
except Exception as e:
    print(f"❌ Prediction failed: {e}")
    import traceback
    traceback.print_exc()


# =======================================================================
# Post-Training: Export for Snapdragon Deployment
# =======================================================================

# After training completes, add this:

print("\n" + "="*70)
print("🚀 Exporting optimized models for Snapdragon X Elite...")
print("="*70)

# 1. TFLite INT8 (Best for NPU)
convert_to_tflite_quantized(model, 'emotion_model_int8.tflite')

# 2. TFLite FP16 (Good balance)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_fp16 = converter.convert()
with open('emotion_model_fp16.tflite', 'wb') as f:
    f.write(tflite_fp16)
print(f"✅ FP16 model saved: emotion_model_fp16.tflite ({len(tflite_fp16)/1024:.2f} KB)")

# 3. ONNX (for cross-platform)
try:
    import tf2onnx
    spec = (tf.TensorSpec((None, 48, 48, 1), tf.float32, name="input"),)
    model_proto, _ = tf2onnx.convert.from_keras(
        model, 
        input_signature=spec, 
        opset=13, 
        output_path="emotion_model.onnx"
    )
    print("✅ ONNX model saved: emotion_model.onnx")
except:
    print("⚠️ ONNX export skipped (install: pip install tf2onnx)")

print("\n" + "="*70)
print("📊 Model Export Summary:")
print("="*70)
print("1. emotion_model_int8.tflite  → Best for Snapdragon NPU (fastest)")
print("2. emotion_model_fp16.tflite  → Good balance (fast + accurate)")
print("3. emotion_model.onnx         → Cross-platform deployment")
print("="*70)